# Python: Average Potential Outcome (APO) Models

In this example, we illustrate how the [DoubleML](https://docs.doubleml.org/stable/index.html) package can be used to estimate average potential outcomes (APOs) in an interactive regression model (see [DoubleMLIRM](https://docs.doubleml.org/stable/guide/models.html#binary-interactive-regression-model-irm)).

The goal is to estimate the average potential outcome

 $$\theta_0 =\mathbb{E}[Y(d)]$$

for a given treatment level $d$ and and discrete valued treatment $D$.

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

import doubleml as dml
from doubleml.irm.datasets import make_irm_data_discrete_treatments

## Data Generating Process (DGP)

At first, let us generate data according to the [make_irm_data_discrete_treatments](https://docs.doubleml.org/dev/api/generated/doubleml.irm.datasets.make_irm_data_discrete_treatments.html#doubleml.irm.datasets.make_irm_data_discrete_treatments) data generating process. The process generates data with a continuous
treatment variable and contains the true individual treatment effects (ITEs) with respect to option of not getting treated.

According to the continuous treatment variable, the treatment is discretized into multiple levels, based on quantiles. Using the *oracle* ITEs, enables the comparison to the true APOs and averate treatment effects (ATEs) for the different levels of the treatment variable.

**Remark:** The average potential outcome model does not require an underlying continuous treatment variable. The model will work identically if the treatment variable is discrete by design.

In [ ]:
# Parameters
n_obs = 3000
n_levels = 5
linear = True
n_rep = 10

np.random.seed(42)
data_apo = make_irm_data_discrete_treatments(n_obs=n_obs,n_levels=n_levels, linear=linear)

y0 = data_apo['oracle_values']['y0']
cont_d = data_apo['oracle_values']['cont_d']
ite = data_apo['oracle_values']['ite']
d = data_apo['d']
potential_level = data_apo['oracle_values']['potential_level']
level_bounds = data_apo['oracle_values']['level_bounds']

average_ites = np.full(n_levels + 1, np.nan)
apos = np.full(n_levels + 1, np.nan)
mid_points = np.full(n_levels, np.nan)

for i in range(n_levels + 1):
    average_ites[i] = np.mean(ite[d == i]) * (i > 0)
    apos[i] = np.mean(y0) + average_ites[i]

print(f"Average Individual effects in each group:\n{np.round(average_ites,2)}\n")
print(f"Average Potential Outcomes in each group:\n{np.round(apos,2)}\n")
print(f"Levels and their counts:\n{np.unique(d, return_counts=True)}")

To better grasp the distribution of the treatment effects, let us plot the true APOs and ATEs.

In [ ]:
# Get a colorblind-friendly palette
palette = sns.color_palette("colorblind")

df = pd.DataFrame({'cont_d': cont_d, 'ite': ite})
df_sorted = df.sort_values('cont_d')

mid_points = np.full(n_levels, np.nan)
for i in range(n_levels):
    mid_points[i] = (level_bounds[i] + level_bounds[i + 1]) / 2

df_apos = pd.DataFrame({'mid_points': mid_points, 'treatment effects': apos[1:] - apos[0]})

# Create the primary plot with scatter and line plots
fig, ax1 = plt.subplots()

sns.lineplot(data=df_sorted, x='cont_d', y='ite', color=palette[0], label='ITE', ax=ax1)
sns.scatterplot(data=df_apos, x='mid_points', y='treatment effects', color=palette[1], label='Grouped Treatment Effects', ax=ax1)

# Add vertical dashed lines at level_bounds
for bound in level_bounds:
    ax1.axvline(x=bound, color='grey', linestyle='--', alpha=0.7)

ax1.set_title('Grouped Effects vs. Continuous Treatment')
ax1.set_xlabel('Continuous Treatment')
ax1.set_ylabel('Effects')

# Create a secondary y-axis for the histogram
ax2 = ax1.twinx()

# Plot the histogram on the secondary y-axis
ax2.hist(df_sorted['cont_d'], bins=30, alpha=0.3, weights=np.ones_like(df_sorted['cont_d']) / len(df_sorted['cont_d']), color=palette[2])
ax2.set_ylabel('Density')

# Make sure the legend includes all plots
lines, labels = ax1.get_legend_handles_labels()
ax1.legend(lines, labels, loc='upper left')

plt.show()

As for all [DoubleML](https://docs.doubleml.org/stable/index.html) models, we specify a [DoubleMLData](https://docs.doubleml.org/stable/api/generated/doubleml.data.DoubleMLData.html) object to handle the data.

In [ ]:
y = data_apo['y']
x = data_apo['x']
d = data_apo['d']
df_apo = pd.DataFrame(
    np.column_stack((y, d, x)),
    columns=['y', 'd'] + ['x' + str(i) for i in range(data_apo['x'].shape[1])]
)

dml_data = dml.DoubleMLData(df_apo, 'y', 'd')
print(dml_data)

## Single Average Potential Outcome Models (APO)

Further, we have to specify machine learning algorithms. As in the [DoubleMLIRM](https://docs.doubleml.org/stable/guide/models.html#interactive-regression-models-irm) model, we have to set ``ml_m`` as a classifier and ``ml_g`` as a regressor (since the outcome is continuous). As in the 
[DoubleMLIRM](https://docs.doubleml.org/stable/guide/models.html#interactive-regression-models-irm) model, the classifier ``ml_m`` is used to estimate the conditional probability of receiving treatment level $d$ given the covariates $X$

$$m_{0,d}(X) = \mathbb{E}[1\{D=d\}|X]$$

and the regressor ``ml_g`` is used to estimate the conditional expectation of the outcome $Y$ given the covariates $X$ and the treatment $D$

$$g_{0}(D, X) = \mathbb{E}[Y|X,D].$$

As the DGP is linear we will use a linear regression model for the regressor and a logistic regression model for the classifier.

In [ ]:
ml_g = LinearRegression()
ml_m = LogisticRegression()

Further, the [DoubleMLAPO](https://docs.doubleml.org/dev/api/generated/doubleml.irm.DoubleMLAPO.html) model requires a specification of the treatment level $a$ for which the APOs should be estimated. In this example, we will loop over all treatment levels.

In [ ]:
np.random.seed(42)

treatment_levels = np.unique(d)
thetas = np.full(n_levels + 1, np.nan)
ci = np.full((n_levels + 1, 2), np.nan)

for i_level, treatment_level in enumerate(treatment_levels):
    dml_obj = dml.DoubleMLAPO(
        dml_data,
        ml_g,
        ml_m,
        treatment_level=treatment_level,
        n_rep=n_rep,
    )

    dml_obj.fit()

    thetas[i_level] = dml_obj.coef[0]
    ci[i_level, :] = dml_obj.confint(level=0.95).values

# combine results
df_apo_ci = pd.DataFrame(
    {'treatment_level': treatment_levels,
     'apo': apos,
     'theta': thetas,
     'ci_lower': ci[:, 0],
     'ci_upper': ci[:, 1]}
)

df_apo_ci

The tables above displays the estimated values in the ``theta`` column and the corresponding oracle values in the ``apo`` column. 

Again, let us summarize the results in a plot of the APOs with confidence intervals.

In [ ]:
# Plotting
plt.figure(figsize=(10, 6))
# Plot Estimate with 95% CI
plt.errorbar(df_apo_ci['treatment_level'], df_apo_ci['theta'], 
             yerr=[df_apo_ci['theta'] - df_apo_ci['ci_lower'], df_apo_ci['ci_upper'] - df_apo_ci['theta']], 
             fmt='o', capsize=5, capthick=2, ecolor=palette[1], color=palette[0], label='Estimate with 95% CI', zorder=2)
# Plot APO as a scatter plot, with zorder set to 2 to be in front
plt.scatter(df_apo_ci['treatment_level'], df_apo_ci['apo'], color=palette[2], label='APO', marker='d', zorder=3)

plt.title('Estimated APO, Theta, and 95% Confidence Interval by Treatment Level')
plt.xlabel('Treatment Level')
plt.ylabel('Value')
plt.xticks(df_apo_ci['treatment_level'])
plt.legend()
plt.grid(True)
plt.show()

## Multiple Average Potential Outcome Models (APOS)

Instead of looping over different treatment levels, one can directly use the [DoubleMLAPOS](https://docs.doubleml.org/dev/api/generated/doubleml.irm.DoubleMLAPOS.html#doubleml.irm.DoubleMLAPOS) model which internally combines multiple [DoubleMLAPO](https://docs.doubleml.org/dev/api/generated/doubleml.irm.DoubleMLAPO.html) models. An advantage of this approach is that the model can be parallelized, create joint confidence intervals and allow for a comparison between the average potential outcome levels.

### Average Potential Outcome (APOs)

As before, we just have to specify the machine learning algorithms and the treatment levels for which the APOs should be estimated.

In [ ]:
dml_obj = dml.DoubleMLAPOS(
    dml_data,
    ml_g,
    ml_m,
    treatment_levels=treatment_levels,
    n_rep=n_rep,
)

dml_obj.fit()

ci_pointwise = dml_obj.confint(level=0.95)

df_apos_ci = pd.DataFrame(
    {'treatment_level': treatment_levels,
     'apo': apos,
     'theta': thetas,
     'ci_lower': ci_pointwise.values[:, 0],
     'ci_upper': ci_pointwise.values[:, 1]}
)

df_apos_ci

Again, let us summarize the results in a plot.

In [ ]:
# Plotting
plt.figure(figsize=(10, 6))
# Plot Estimate with 95% CI
plt.errorbar(df_apos_ci['treatment_level'], df_apos_ci['theta'], 
             yerr=[df_apos_ci['theta'] - df_apos_ci['ci_lower'], df_apos_ci['ci_upper'] - df_apos_ci['theta']], 
             fmt='o', capsize=5, capthick=2, ecolor=palette[1], color=palette[0], label='Estimate with 95% CI', zorder=2)
# Plot APO as a scatter plot, with zorder set to 2 to be in front
plt.scatter(df_apos_ci['treatment_level'], df_apos_ci['apo'], color=palette[2], label='APO', marker='d', zorder=3)

plt.title('Estimated APO, Theta, and 95% Confidence Interval by Treatment Level')
plt.xlabel('Treatment Level')
plt.ylabel('Value')
plt.xticks(df_apos_ci['treatment_level'])
plt.legend()
plt.grid(True)
plt.show()

For joint confidence intervals, the ``bootstrap`` method can be used.

In [ ]:
dml_obj.bootstrap(n_rep_boot=2000)
ci_joint = dml_obj.confint(level=0.95, joint=True)

ci_joint

## Sensitivity Analysis

For [DoubleMLAPO](https://docs.doubleml.org/dev/api/generated/doubleml.irm.DoubleMLAPO.html) and [DoubleMLAPOS](https://docs.doubleml.org/dev/api/generated/doubleml.irm.DoubleMLAPOS.html#doubleml.irm.DoubleMLAPOS) model all methods for sensitivity analysis are available.

In [ ]:
dml_obj.sensitivity_analysis()
print(dml_obj.sensitivity_summary)

Additionally, ``sensitvity_benchmark`` can be used. In this example we benchmark covariate ``x4`` which does not affect treatment $D$ or outcome $Y$.

In [ ]:
dml_obj.sensitivity_benchmark(benchmarking_set=['x4'])

 For more details on the sensitivity analysis, please refer to the [User Guide](https://docs.doubleml.org/stable/guide/sensitivity.html).

### Causal Contrasts

The [DoubleMLAPOS](https://docs.doubleml.org/dev/api/generated/doubleml.irm.DoubleMLAPOS.html#doubleml.irm.DoubleMLAPOS) model also allows for the estimation of causal contrasts. 
The contrast is defined as the difference in the average potential outcomes between the treatment levels $d_i$ and $d_j$ where

$$ \theta_{0,ij} = \mathbb{E}[Y(d_i)] - \mathbb{E}[Y(d_{j})]$$

and will be calculated for all defined treatment levels $i$ and reference levels $j$.

In this example, we will estimate the causal contrast between the treatment level $0$ and all other treatment levels, as the treatment level $0$ corresponds to no treatment at all whereas the the other levels are based on the treatment dosage.

Therefore we have to specify ``reference_levels=0``.

In [ ]:
causal_contrast_model = dml_obj.causal_contrast(reference_levels=0)
print(causal_contrast_model.summary)

Finally, let us summarize the results in a plot.

In [ ]:
ates = causal_contrast_model.thetas
ci_ates = causal_contrast_model.confint(level=0.95)

df_ates = pd.DataFrame(
    {'treatment_level': treatment_levels[1:],
     'ate': ates,
     'ci_lower': ci_ates.iloc[:, 0].values,
     'ci_upper': ci_ates.iloc[:, 1].values}
)

# Plotting
plt.figure(figsize=(10, 6))
# Plot Estimate with 95% CI
plt.errorbar(df_ates['treatment_level'], df_ates['ate'],
             yerr=[df_ates['ate'] - df_ates['ci_lower'], df_ates['ci_upper'] - df_ates['ate']], 
             fmt='o', capsize=5, capthick=2, ecolor=palette[1], color=palette[0], label='Estimate with 95% CI', zorder=2)
# Plot APO as a scatter plot, with zorder set to 2 to be in front
plt.scatter(df_apos_ci['treatment_level'][1:], average_ites[1:], color=palette[2], label='ATE', marker='d', zorder=3)

plt.title('Estimated ATE, Theta, and 95% Confidence Interval by Treatment Level')
plt.xlabel('Treatment Level')
plt.ylabel('Value')
plt.xticks(df_apos_ci['treatment_level'])
plt.legend()
plt.grid(True)
plt.show()

The methods ``sensitivity_analysis`` and ``sensitivity_plot`` are also available for the causal contrasts.

In [ ]:
causal_contrast_model.sensitivity_analysis()
print(causal_contrast_model.sensitivity_summary)

As an example see the ``sensitivity_plot`` for the first causal contrast ``1.0 vs 0.0``.

In [ ]:
causal_contrast_model.sensitivity_plot(idx_treatment=0)

Benchmarking is not available for causal contrasts.